In [ ]:
import torch
from sklearn.preprocessing import minmax_scale
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import optuna
from torch.utils.data import DataLoader
from torch import nn

from utils_training import train_and_evaluate
from utils_training import PrepareData
from utils_training import RNNModel
from utils_preprocessing import filter_emg
from utils_preprocessing import resample_by_linspace
from utils_training import train_test
from utils_metrics import get_error_metrics

print(torch.cuda.is_available())
print(torch.version.cuda)
plt.ion()
plt.isinteractive()

True
12.1


True

In [ ]:
data = pd.read_pickle('dataset_siat_withStatus_allGroups.pkl')
print(data.shape, data.shape[0]==400) # mentioned in paper
data.head(3)

(400, 33) True


,SubjID,Gender,Age,BH,BM,Time,Kinematic_left_hip_adduction_angle,Kinematic_left_hip_flexion_angle,Kinematic_left_knee_flexion_angle,Kinematic_left_ankle_flexion_angle,...,sEMG_rectus_femoris,sEMG_vastus_medialis,sEMG_semimembranosus,sEMG_upper_tibialis_anterior,sEMG_lower_tibialis_anterior,sEMG_lateral_gastrocnemius,sEMG_medial_gastrocnemius,sEMG_soleus,Status,Group
0,Sub01,male,22,1750,68.7,"[0.16041666553125, 0.160937498828125, 0.161458...","[-2.29510279603125, -2.27232552375, -2.2495482...","[28.88873772371876, 28.863172175750005, 28.837...","[23.31136918525, 23.316881248000005, 23.322393...","[1.9352270099375, 1.9513703325, 1.967513655062...",...,"[-0.009765625, -0.01190185546875, -0.011291503...","[-0.0115966796875, -0.0067138671875, -0.002136...","[-0.0274658203125, -0.0347900390625, -0.042114...","[0.00732421875, 0.00732421875, 0.0070190429687...","[-0.0262451171875, -0.0286865234375, -0.028381...","[0.06378173828125, 0.06378173828125, 0.0650024...","[0.00091552734375, 0.00091552734375, 0.0006103...","[0.00213623046875, 0.0006103515625, 0.0, 0.001...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1
1,Sub01,male,22,1750,68.7,"[1.585937499296875, 1.58645833259375, 1.586979...","[-1.84181140909375, -1.813811492359375, -1.785...","[32.113622010250005, 32.094555400875, 32.07548...","[24.798581518187504, 24.826332252218755, 24.85...","[2.04114409696875, 2.055421115296875, 2.069698...",...,"[-0.00823974609375, -0.00946044921875, -0.0122...","[-0.0341796875, -0.030517578125, -0.0207519531...","[0.020751953125, 0.020751953125, 0.01861572265...","[0.0494384765625, 0.0421142578125, 0.050048828...","[-0.07476806640625, -0.07049560546875, -0.0711...","[-0.10040283203125, -0.09521484375, -0.0912475...","[-0.00213623046875, -0.0006103515625, -0.00122...","[-0.0091552734375, -0.010986328125, -0.0091552...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",2
2,Sub01,male,22,1750,68.7,"[3.046874997828125, 3.047395831265625, 3.04791...","[-1.1017423845, -1.0779376805, -1.0541329765, ...","[29.21583017823438, 29.192614021312504, 29.169...","[22.31687331540625, 22.335616773875003, 22.354...","[0.97311782259375, 0.991092862625, 1.009067902...",...,"[-0.015869140625, -0.0189208984375, -0.0213623...","[-0.02349853515625, -0.0274658203125, -0.02899...","[-0.0018310546875, -0.00091552734375, 0.001525...","[0.01678466796875, 0.01251220703125, 0.0061035...","[-0.0360107421875, -0.0433349609375, -0.044860...","[-0.01678466796875, -0.01129150390625, -0.0061...","[-0.00396728515625, -0.00213623046875, 0.00061...","[-0.02655029296875, -0.02593994140625, -0.0247...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",3


In [ ]:
data_cols = [c for c in data.columns if (('sEMG' in c) or ('Kine' in c))]
info_columns = ['SubjID', 'Gender', 'Age', 'BH', 'BM']

data[["Age", "BH", "BM"]] = data[["Age", "BH", "BM"]].apply(pd.to_numeric)

(200, 33)

In [ ]:
# Select left-right ordered steps
data = data[data.Group<6]

# Select only swing phase 
for df in data.iterrows():
  d_loc = df[0]
  df = df[1]
  stance_idx = df['Status']<4
  # print(d_loc, len(df['Status']), sum(stance_idx))
  for c in data_cols:
    data.at[d_loc, c] = df[c][stance_idx]

data['Status'] = data['Status'].apply(lambda x: x[x<4])

In [ ]:
data_selected = data
print('Num of Subj:', len(data_selected.SubjID.unique()))
data_selected.head(3)

Num of Subj: 40


,SubjID,Gender,Age,BH,BM,Time,Kinematic_left_hip_adduction_angle,Kinematic_left_hip_flexion_angle,Kinematic_left_knee_flexion_angle,Kinematic_left_ankle_flexion_angle,...,sEMG_rectus_femoris,sEMG_vastus_medialis,sEMG_semimembranosus,sEMG_upper_tibialis_anterior,sEMG_lower_tibialis_anterior,sEMG_lateral_gastrocnemius,sEMG_medial_gastrocnemius,sEMG_soleus,Status,Group
0,Sub01,male,22,1750,68.7,"[0.16041666553125, 0.160937498828125, 0.161458...","[-2.29510279603125, -2.27232552375, -2.2495482...","[28.88873772371876, 28.863172175750005, 28.837...","[23.31136918525, 23.316881248000005, 23.322393...","[1.9352270099375, 1.9513703325, 1.967513655062...",...,"[-0.009765625, -0.01190185546875, -0.011291503...","[-0.0115966796875, -0.0067138671875, -0.002136...","[-0.0274658203125, -0.0347900390625, -0.042114...","[0.00732421875, 0.00732421875, 0.0070190429687...","[-0.0262451171875, -0.0286865234375, -0.028381...","[0.06378173828125, 0.06378173828125, 0.0650024...","[0.00091552734375, 0.00091552734375, 0.0006103...","[0.00213623046875, 0.0006103515625, 0.0, 0.001...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1
1,Sub01,male,22,1750,68.7,"[1.585937499296875, 1.58645833259375, 1.586979...","[-1.84181140909375, -1.813811492359375, -1.785...","[32.113622010250005, 32.094555400875, 32.07548...","[24.798581518187504, 24.826332252218755, 24.85...","[2.04114409696875, 2.055421115296875, 2.069698...",...,"[-0.00823974609375, -0.00946044921875, -0.0122...","[-0.0341796875, -0.030517578125, -0.0207519531...","[0.020751953125, 0.020751953125, 0.01861572265...","[0.0494384765625, 0.0421142578125, 0.050048828...","[-0.07476806640625, -0.07049560546875, -0.0711...","[-0.10040283203125, -0.09521484375, -0.0912475...","[-0.00213623046875, -0.0006103515625, -0.00122...","[-0.0091552734375, -0.010986328125, -0.0091552...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",2
2,Sub01,male,22,1750,68.7,"[3.046874997828125, 3.047395831265625, 3.04791...","[-1.1017423845, -1.0779376805, -1.0541329765, ...","[29.21583017823438, 29.192614021312504, 29.169...","[22.31687331540625, 22.335616773875003, 22.354...","[0.97311782259375, 0.991092862625, 1.009067902...",...,"[-0.015869140625, -0.0189208984375, -0.0213623...","[-0.02349853515625, -0.0274658203125, -0.02899...","[-0.0018310546875, -0.00091552734375, 0.001525...","[0.01678466796875, 0.01251220703125, 0.0061035...","[-0.0360107421875, -0.0433349609375, -0.044860...","[-0.01678466796875, -0.01129150390625, -0.0061...","[-0.00396728515625, -0.00213623046875, 0.00061...","[-0.02655029296875, -0.02593994140625, -0.0247...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",3


In [ ]:
emg_cols = [c for c in data_selected.columns if ('sEMG_' in c)]
kinetic_cols = [c for c in data_selected.columns if ('Kinetic_left' in c)]
kinematic_cols = [c for c in data_selected.columns if ('Kinematic_left' in c)]

emg_cols_preprocessed       = [c+'_preprocessed' for c in emg_cols]
kinetic_cols_preprocessed   = [c+'_preprocessed' for c in kinetic_cols]
kinematic_cols_preprocessed = [c+'_preprocessed' for c in kinematic_cols]

it's ok


Pre-processing

In [ ]:
sequence_len = 60

pd.set_option('mode.chained_assignment', None)

# EMG pre-processing
for c in emg_cols:
    data_selected[c+'_filtered'] = data_selected.apply(lambda x: filter_emg(emg=x[c], low_pass=10, sfreq=1926, high_band=5, low_band=400)[2], axis=1)

emg_cols_filtered = [c+'_filtered' for c in emg_cols]
for c in emg_cols_filtered:
    data_selected[c+'_resampled'] = data_selected.apply(lambda x: resample_by_linspace(x[c], sequence_len), axis=1)

emg_cols_resampled = [c+'_resampled' for c in emg_cols_filtered]
for c in emg_cols_resampled:
    data_selected[c+'_scaled'] = data_selected.apply(lambda x: minmax_scale(x[c], feature_range=(0, 1)), axis=1)

# Torque pre-processing
for c in kinetic_cols:
    # data_selected[c] = data_selected[c]/data_selected['BM']
    data_selected[c] = data_selected[c]

for c in kinetic_cols:
    data_selected[c+'_resampled'] = data_selected.apply(lambda x: resample_by_linspace(x[c], sequence_len), axis=1)

kinetic_cols_resampled = [c+'_resampled' for c in kinetic_cols]
for c in kinetic_cols_resampled:
    data_selected[c+'_scaled'] = data_selected.apply(lambda x: minmax_scale(x[c], feature_range=(0, 1)), axis=1)

# Angle pre-processing
for c in kinematic_cols:
    data_selected[c+'_resampled'] = data_selected.apply(lambda x: resample_by_linspace(x[c], sequence_len), axis=1)

kinematic_cols_resampled = [c+'_resampled' for c in kinematic_cols]
for c in kinematic_cols_resampled:
    data_selected[c+'_scaled'] = data_selected.apply(lambda x: minmax_scale(x[c], feature_range=(0, 1)), axis=1)

# Status pre-processing
c = 'Status'
data_selected[c+'_resampled'] = data_selected.apply(lambda x: resample_by_linspace(x[c], sequence_len), axis=1)

pd.set_option('mode.chained_assignment', 'warn')

In [ ]:
emg_cols_preprocessed_selected_cols = [c for c in data_selected.columns if(('sEMG_' in c) and ('_scaled' in c))]
kinematic_cols_preprocessed_selected_cols = [c for c in data_selected.columns if(('Kinematic_' in c) and ('_scaled' in c))]
kinetic_cols_preprocessed_selected_cols = [c for c in data_selected.columns if(('Kinetic_' in c) and ('_scaled' in c))]

data_selected[emg_cols_preprocessed]       = data_selected[emg_cols_preprocessed_selected_cols].copy()
data_selected[kinetic_cols_preprocessed]   = data_selected[kinetic_cols_preprocessed_selected_cols].copy()
data_selected[kinematic_cols_preprocessed] = data_selected[kinematic_cols_preprocessed_selected_cols].copy()

print(len(data_selected.SubjID.unique()))
data_selected.shape

40


(200, 94)

In [ ]:
kinetic_cols_preprocessed = [ 'Kinetic_left_hip_flexion_torque_preprocessed',
                              'Kinetic_left_knee_flexion_torque_preprocessed',
                              'Kinetic_left_ankle_flexion_torque_preprocessed']

Find Best Hyperparameters

In [ ]:
def objective_both(trial):
    in_cols = emg_cols_preprocessed + kinematic_cols_preprocessed
    out_cols = kinetic_cols_preprocessed

    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True),
        'hidden_dim': trial.suggest_int("hidden_dim", 10, 100),
        'layer_dim': trial.suggest_int("layer_dim", 1, 4),
        'num_epoch': 200
    }

    input_dim = len(in_cols)    # input dimension
    output_dim = len(out_cols)   # output dimension
    hidden_dim = params['hidden_dim']
    layer_dim = params['layer_dim']

    # Configuration options
    k_folds = 4
    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    subjects = data_selected.SubjID.unique()

    total_loss_all_folds = 0

    # K-fold Cross Validation model evaluation
    for fold, (train_subjs, test_subjs) in enumerate(kfold.split(subjects)):
        # Print
        print('--------------------------------')
        print(f'FOLD {fold}  Test Subjects: {subjects[test_subjs]}')

        train_idxs = data_selected[data_selected.SubjID.isin(subjects[train_subjs])].index
        test_idxs = data_selected[data_selected.SubjID.isin(subjects[test_subjs])].index

        train_df = data_selected.loc[train_idxs]
        test_df = data_selected.loc[test_idxs]

        ds_train = PrepareData(train_df, in_cols, out_cols)
        train_loader = DataLoader(ds_train, batch_size=8, shuffle=True, drop_last=True)

        ds_valid = PrepareData(test_df, in_cols, out_cols)
        valid_loader = DataLoader(ds_valid, batch_size=8, shuffle=True, drop_last=True)

        model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
        if torch.cuda.is_available():
            model = model.cuda()

        _, total_loss, _ = train_and_evaluate(params, model, train_loader, valid_loader)

        total_loss_all_folds = total_loss_all_folds + total_loss

    return total_loss_all_folds


study_both = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())

study_both.optimize(objective_both, n_trials=20)

study_both_df = study_both.trials_dataframe()
study_both_df.sort_values('value')

[I 2024-04-21 10:41:43,254] A new study created in memory with name: no-name-f0a895c6-8daf-473b-a861-d4ce0f98e81f


--------------------------------
FOLD 0  Test Subjects: ['Sub05' 'Sub07' 'Sub13' 'Sub16' 'Sub17' 'Sub20' 'Sub27' 'Sub28' 'Sub38'
 'Sub40']
--------------------------------
FOLD 1  Test Subjects: ['Sub01' 'Sub09' 'Sub10' 'Sub14' 'Sub18' 'Sub25' 'Sub26' 'Sub32' 'Sub34'
 'Sub35']
--------------------------------
FOLD 2  Test Subjects: ['Sub02' 'Sub03' 'Sub04' 'Sub06' 'Sub12' 'Sub22' 'Sub30' 'Sub31' 'Sub36'
 'Sub37']
--------------------------------
FOLD 3  Test Subjects: ['Sub08' 'Sub11' 'Sub15' 'Sub19' 'Sub21' 'Sub23' 'Sub24' 'Sub29' 'Sub33'
 'Sub39']


[I 2024-04-21 10:42:50,286] Trial 0 finished with value: 0.047740412643179304 and parameters: {'learning_rate': 0.00026719804507864704, 'weight_decay': 6.080110121696446e-05, 'hidden_dim': 29, 'layer_dim': 2}. Best is trial 0 with value: 0.047740412643179304.


--------------------------------
FOLD 0  Test Subjects: ['Sub05' 'Sub07' 'Sub13' 'Sub16' 'Sub17' 'Sub20' 'Sub27' 'Sub28' 'Sub38'
 'Sub40']
--------------------------------
FOLD 1  Test Subjects: ['Sub01' 'Sub09' 'Sub10' 'Sub14' 'Sub18' 'Sub25' 'Sub26' 'Sub32' 'Sub34'
 'Sub35']
--------------------------------
FOLD 2  Test Subjects: ['Sub02' 'Sub03' 'Sub04' 'Sub06' 'Sub12' 'Sub22' 'Sub30' 'Sub31' 'Sub36'
 'Sub37']
--------------------------------
FOLD 3  Test Subjects: ['Sub08' 'Sub11' 'Sub15' 'Sub19' 'Sub21' 'Sub23' 'Sub24' 'Sub29' 'Sub33'
 'Sub39']


[I 2024-04-21 10:43:50,349] Trial 1 finished with value: 0.0503727983062466 and parameters: {'learning_rate': 0.0004672951467159583, 'weight_decay': 0.0009934635191191702, 'hidden_dim': 57, 'layer_dim': 2}. Best is trial 0 with value: 0.047740412643179304.


...

--------------------------------
FOLD 0  Test Subjects: ['Sub05' 'Sub07' 'Sub13' 'Sub16' 'Sub17' 'Sub20' 'Sub27' 'Sub28' 'Sub38'
 'Sub40']
--------------------------------
FOLD 1  Test Subjects: ['Sub01' 'Sub09' 'Sub10' 'Sub14' 'Sub18' 'Sub25' 'Sub26' 'Sub32' 'Sub34'
 'Sub35']
--------------------------------
FOLD 2  Test Subjects: ['Sub02' 'Sub03' 'Sub04' 'Sub06' 'Sub12' 'Sub22' 'Sub30' 'Sub31' 'Sub36'
 'Sub37']
--------------------------------
FOLD 3  Test Subjects: ['Sub08' 'Sub11' 'Sub15' 'Sub19' 'Sub21' 'Sub23' 'Sub24' 'Sub29' 'Sub33'
 'Sub39']


[I 2024-04-21 11:00:49,577] Trial 18 finished with value: 0.04893593412513535 and parameters: {'learning_rate': 0.0010855672868988453, 'weight_decay': 0.0005269973590778287, 'hidden_dim': 68, 'layer_dim': 2}. Best is trial 17 with value: 0.0448272351641208.


--------------------------------
FOLD 0  Test Subjects: ['Sub05' 'Sub07' 'Sub13' 'Sub16' 'Sub17' 'Sub20' 'Sub27' 'Sub28' 'Sub38'
 'Sub40']
--------------------------------
FOLD 1  Test Subjects: ['Sub01' 'Sub09' 'Sub10' 'Sub14' 'Sub18' 'Sub25' 'Sub26' 'Sub32' 'Sub34'
 'Sub35']
--------------------------------
FOLD 2  Test Subjects: ['Sub02' 'Sub03' 'Sub04' 'Sub06' 'Sub12' 'Sub22' 'Sub30' 'Sub31' 'Sub36'
 'Sub37']
--------------------------------
FOLD 3  Test Subjects: ['Sub08' 'Sub11' 'Sub15' 'Sub19' 'Sub21' 'Sub23' 'Sub24' 'Sub29' 'Sub33'
 'Sub39']


[I 2024-04-21 11:01:50,454] Trial 19 finished with value: 0.04706553905270994 and parameters: {'learning_rate': 0.0011179171697815891, 'weight_decay': 0.0008159962357433759, 'hidden_dim': 50, 'layer_dim': 2}. Best is trial 17 with value: 0.0448272351641208.


,number,value,datetime_start,datetime_complete,duration,params_hidden_dim,params_layer_dim,params_learning_rate,params_weight_decay,state
17,17,0.044827,2024-04-21 10:58:52.236953,2024-04-21 10:59:51.347057,0 days 00:00:59.110104,44,2,0.001318,0.000487,COMPLETE
13,13,0.045668,2024-04-21 10:55:23.241109,2024-04-21 10:56:15.183263,0 days 00:00:51.942154,44,1,0.002190,0.000187,COMPLETE
11,11,0.046060,2024-04-21 10:53:30.319361,2024-04-21 10:54:22.730433,0 days 00:00:52.411072,39,1,0.004014,0.000140,COMPLETE
15,15,0.046098,2024-04-21 10:57:07.386266,2024-04-21 10:57:59.963614,0 days 00:00:52.577348,43,1,0.003622,0.000216,COMPLETE
14,14,0.046298,2024-04-21 10:56:15.184925,2024-04-21 10:57:07.384531,0 days 00:00:52.199606,53,1,0.002540,0.000224,COMPLETE
19,19,0.047066,2024-04-21 11:00:49.581066,2024-04-21 11:01:50.454291,0 days 00:01:00.873225,50,2,0.001118,0.000816,COMPLETE
0,0,0.047740,2024-04-21 10:41:43.260917,2024-04-21 10:42:50.286225,0 days 00:01:07.025308,29,2,0.000267,0.000061,COMPLETE
8,8,0.047996,2024-04-21 10:50:18.893966,2024-04-21 10:51:11.749563,0 days 00:00:52.855597,43,1,0.009590,0.000476,COMPLETE
16,16,0.048846,2024-04-21 10:57:59.966961,2024-04-21 10:58:52.229521,0 days 00:00:52.262560,59,1,0.001805,0.000026,COMPLETE
18,18,0.048936,2024-04-21 10:59:51.348893,2024-04-21 11:00:49.577399,0 days 00:00:58.228506,68,2,0.001086,0.000527,COMPLETE


In [ ]:
def objective_emg(trial):
    in_cols = emg_cols_preprocessed
    out_cols = kinetic_cols_preprocessed

    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True),
        'hidden_dim': trial.suggest_int("hidden_dim", 10, 100),
        'layer_dim': trial.suggest_int("layer_dim", 1, 4),
        'num_epoch': 200
    }

    input_dim = len(in_cols)    # input dimension
    output_dim = len(out_cols)   # output dimension
    hidden_dim = params['hidden_dim']
    layer_dim = params['layer_dim']

    # Configuration options
    k_folds = 4
    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    subjects = data_selected.SubjID.unique()

    total_loss_all_folds = 0

    # K-fold Cross Validation model evaluation
    for fold, (train_subjs, test_subjs) in enumerate(kfold.split(subjects)):
        # Print
        print('--------------------------------')
        print(f'FOLD {fold}  Test Subjects: {subjects[test_subjs]}')

        train_idxs = data_selected[data_selected.SubjID.isin(subjects[train_subjs])].index
        test_idxs = data_selected[data_selected.SubjID.isin(subjects[test_subjs])].index

        train_df = data_selected.loc[train_idxs]
        test_df = data_selected.loc[test_idxs]

        ds_train = PrepareData(train_df, in_cols, out_cols)
        train_loader = DataLoader(ds_train, batch_size=8, shuffle=True, drop_last=True)

        ds_valid = PrepareData(test_df, in_cols, out_cols)
        valid_loader = DataLoader(ds_valid, batch_size=8, shuffle=True, drop_last=True)

        model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
        if torch.cuda.is_available():
            model = model.cuda()

        _, total_loss, _ = train_and_evaluate(params, model, train_loader, valid_loader)

        total_loss_all_folds = total_loss_all_folds + total_loss

    return total_loss_all_folds


study_emg = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())

study_emg.optimize(objective_emg, n_trials=20)

study_emg_df = study_emg.trials_dataframe()
study_emg_df.sort_values('value')

[I 2024-04-21 11:12:56,775] A new study created in memory with name: no-name-5b27085a-efed-40fc-846b-283641d3d4ce


--------------------------------
FOLD 0  Test Subjects: ['Sub05' 'Sub07' 'Sub13' 'Sub16' 'Sub17' 'Sub20' 'Sub27' 'Sub28' 'Sub38'
 'Sub40']


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


--------------------------------
FOLD 1  Test Subjects: ['Sub01' 'Sub09' 'Sub10' 'Sub14' 'Sub18' 'Sub25' 'Sub26' 'Sub32' 'Sub34'
 'Sub35']


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


...

--------------------------------
FOLD 0  Test Subjects: ['Sub05' 'Sub07' 'Sub13' 'Sub16' 'Sub17' 'Sub20' 'Sub27' 'Sub28' 'Sub38'
 'Sub40']
--------------------------------
FOLD 1  Test Subjects: ['Sub01' 'Sub09' 'Sub10' 'Sub14' 'Sub18' 'Sub25' 'Sub26' 'Sub32' 'Sub34'
 'Sub35']
--------------------------------
FOLD 2  Test Subjects: ['Sub02' 'Sub03' 'Sub04' 'Sub06' 'Sub12' 'Sub22' 'Sub30' 'Sub31' 'Sub36'
 'Sub37']
--------------------------------
FOLD 3  Test Subjects: ['Sub08' 'Sub11' 'Sub15' 'Sub19' 'Sub21' 'Sub23' 'Sub24' 'Sub29' 'Sub33'
 'Sub39']


[I 2024-04-21 11:33:50,450] Trial 18 finished with value: 0.065367319000264 and parameters: {'learning_rate': 0.0005967870843907095, 'weight_decay': 4.433598637561484e-05, 'hidden_dim': 88, 'layer_dim': 3}. Best is trial 16 with value: 0.05145442221934597.


--------------------------------
FOLD 0  Test Subjects: ['Sub05' 'Sub07' 'Sub13' 'Sub16' 'Sub17' 'Sub20' 'Sub27' 'Sub28' 'Sub38'
 'Sub40']
--------------------------------
FOLD 1  Test Subjects: ['Sub01' 'Sub09' 'Sub10' 'Sub14' 'Sub18' 'Sub25' 'Sub26' 'Sub32' 'Sub34'
 'Sub35']
--------------------------------
FOLD 2  Test Subjects: ['Sub02' 'Sub03' 'Sub04' 'Sub06' 'Sub12' 'Sub22' 'Sub30' 'Sub31' 'Sub36'
 'Sub37']
--------------------------------
FOLD 3  Test Subjects: ['Sub08' 'Sub11' 'Sub15' 'Sub19' 'Sub21' 'Sub23' 'Sub24' 'Sub29' 'Sub33'
 'Sub39']


[I 2024-04-21 11:35:02,820] Trial 19 finished with value: 0.07605263808121285 and parameters: {'learning_rate': 0.004001917000336567, 'weight_decay': 0.0021794847939347875, 'hidden_dim': 76, 'layer_dim': 4}. Best is trial 16 with value: 0.05145442221934597.


,number,value,datetime_start,datetime_complete,duration,params_hidden_dim,params_layer_dim,params_learning_rate,params_weight_decay,state
16,16,0.051454,2024-04-21 11:30:25.892891,2024-04-21 11:31:31.397726,0 days 00:01:05.504835,87,3,0.000677,0.000264,COMPLETE
11,11,0.053169,2024-04-21 11:24:26.660944,2024-04-21 11:25:40.492210,0 days 00:01:13.831266,74,4,0.009079,0.000417,COMPLETE
3,3,0.054018,2024-04-21 11:15:51.493005,2024-04-21 11:17:00.900224,0 days 00:01:09.407219,60,3,0.006643,0.000504,COMPLETE
2,2,0.055664,2024-04-21 11:14:50.738546,2024-04-21 11:15:51.488637,0 days 00:01:00.750091,65,2,0.000112,0.000780,COMPLETE
15,15,0.055728,2024-04-21 11:29:13.926585,2024-04-21 11:30:25.890990,0 days 00:01:11.964405,72,4,0.002269,0.000396,COMPLETE
17,17,0.055782,2024-04-21 11:31:31.400458,2024-04-21 11:32:44.157949,0 days 00:01:12.757491,90,4,0.000664,0.000170,COMPLETE
8,8,0.056672,2024-04-21 11:21:09.071875,2024-04-21 11:22:11.570391,0 days 00:01:02.498516,100,2,0.000032,0.000057,COMPLETE
6,6,0.057762,2024-04-21 11:19:03.052924,2024-04-21 11:20:03.043048,0 days 00:00:59.990124,96,2,0.000019,0.000011,COMPLETE
10,10,0.057880,2024-04-21 11:23:12.904725,2024-04-21 11:24:26.659268,0 days 00:01:13.754543,12,4,0.008853,0.000159,COMPLETE
0,0,0.060491,2024-04-21 11:12:56.778243,2024-04-21 11:13:50.407714,0 days 00:00:53.629471,58,1,0.000225,0.000722,COMPLETE


In [ ]:
def objective_kinematics(trial):
    in_cols = kinematic_cols_preprocessed
    out_cols = kinetic_cols_preprocessed

    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True),
        'hidden_dim': trial.suggest_int("hidden_dim", 10, 100),
        'layer_dim': trial.suggest_int("layer_dim", 1, 4),
        'num_epoch': 200
    }

    input_dim = len(in_cols)    # input dimension
    output_dim = len(out_cols)   # output dimension
    hidden_dim = params['hidden_dim']
    layer_dim = params['layer_dim']

    # Configuration options
    k_folds = 4
    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    subjects = data_selected.SubjID.unique()

    total_loss_all_folds = 0

    # K-fold Cross Validation model evaluation
    for fold, (train_subjs, test_subjs) in enumerate(kfold.split(subjects)):
        # Print
        print('--------------------------------')
        print(f'FOLD {fold}  Test Subjects: {subjects[test_subjs]}')

        train_idxs = data_selected[data_selected.SubjID.isin(subjects[train_subjs])].index
        test_idxs = data_selected[data_selected.SubjID.isin(subjects[test_subjs])].index

        train_df = data_selected.loc[train_idxs]
        test_df = data_selected.loc[test_idxs]

        ds_train = PrepareData(train_df, in_cols, out_cols)
        train_loader = DataLoader(ds_train, batch_size=8, shuffle=True, drop_last=True)

        ds_valid = PrepareData(test_df, in_cols, out_cols)
        valid_loader = DataLoader(ds_valid, batch_size=8, shuffle=True, drop_last=True)

        model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
        if torch.cuda.is_available():
            model = model.cuda()

        _, total_loss, _ = train_and_evaluate(params, model, train_loader, valid_loader)

        total_loss_all_folds = total_loss_all_folds + total_loss

    return total_loss_all_folds


study_kinematics = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())

study_kinematics.optimize(objective_kinematics, n_trials=20)

study_kinematics_df = study_kinematics.trials_dataframe()
study_kinematics_df.sort_values('value')

[I 2024-04-21 11:37:35,642] A new study created in memory with name: no-name-1a67cc51-6b59-462d-8145-3e909a88b038


--------------------------------
FOLD 0  Test Subjects: ['Sub05' 'Sub07' 'Sub13' 'Sub16' 'Sub17' 'Sub20' 'Sub27' 'Sub28' 'Sub38'
 'Sub40']
--------------------------------
FOLD 1  Test Subjects: ['Sub01' 'Sub09' 'Sub10' 'Sub14' 'Sub18' 'Sub25' 'Sub26' 'Sub32' 'Sub34'
 'Sub35']
--------------------------------
FOLD 2  Test Subjects: ['Sub02' 'Sub03' 'Sub04' 'Sub06' 'Sub12' 'Sub22' 'Sub30' 'Sub31' 'Sub36'
 'Sub37']
--------------------------------
FOLD 3  Test Subjects: ['Sub08' 'Sub11' 'Sub15' 'Sub19' 'Sub21' 'Sub23' 'Sub24' 'Sub29' 'Sub33'
 'Sub39']


[I 2024-04-21 11:38:35,406] Trial 0 finished with value: 0.06480622927968702 and parameters: {'learning_rate': 3.969785013240933e-05, 'weight_decay': 0.0008585686467030413, 'hidden_dim': 38, 'layer_dim': 2}. Best is trial 0 with value: 0.06480622927968702.


--------------------------------
FOLD 0  Test Subjects: ['Sub05' 'Sub07' 'Sub13' 'Sub16' 'Sub17' 'Sub20' 'Sub27' 'Sub28' 'Sub38'
 'Sub40']
--------------------------------
FOLD 1  Test Subjects: ['Sub01' 'Sub09' 'Sub10' 'Sub14' 'Sub18' 'Sub25' 'Sub26' 'Sub32' 'Sub34'
 'Sub35']
--------------------------------
FOLD 2  Test Subjects: ['Sub02' 'Sub03' 'Sub04' 'Sub06' 'Sub12' 'Sub22' 'Sub30' 'Sub31' 'Sub36'
 'Sub37']
--------------------------------
FOLD 3  Test Subjects: ['Sub08' 'Sub11' 'Sub15' 'Sub19' 'Sub21' 'Sub23' 'Sub24' 'Sub29' 'Sub33'
 'Sub39']


[I 2024-04-21 11:39:48,110] Trial 1 finished with value: 0.04860755281212429 and parameters: {'learning_rate': 0.0012081499823101342, 'weight_decay': 3.2504621405060626e-05, 'hidden_dim': 25, 'layer_dim': 4}. Best is trial 1 with value: 0.04860755281212429.


...

--------------------------------
FOLD 0  Test Subjects: ['Sub05' 'Sub07' 'Sub13' 'Sub16' 'Sub17' 'Sub20' 'Sub27' 'Sub28' 'Sub38'
 'Sub40']
--------------------------------
FOLD 1  Test Subjects: ['Sub01' 'Sub09' 'Sub10' 'Sub14' 'Sub18' 'Sub25' 'Sub26' 'Sub32' 'Sub34'
 'Sub35']
--------------------------------
FOLD 2  Test Subjects: ['Sub02' 'Sub03' 'Sub04' 'Sub06' 'Sub12' 'Sub22' 'Sub30' 'Sub31' 'Sub36'
 'Sub37']
--------------------------------
FOLD 3  Test Subjects: ['Sub08' 'Sub11' 'Sub15' 'Sub19' 'Sub21' 'Sub23' 'Sub24' 'Sub29' 'Sub33'
 'Sub39']


[I 2024-04-21 11:58:03,830] Trial 18 finished with value: 0.05462073006977638 and parameters: {'learning_rate': 0.0006707853222344187, 'weight_decay': 2.659404853256203e-05, 'hidden_dim': 52, 'layer_dim': 3}. Best is trial 14 with value: 0.04780314459155004.


--------------------------------
FOLD 0  Test Subjects: ['Sub05' 'Sub07' 'Sub13' 'Sub16' 'Sub17' 'Sub20' 'Sub27' 'Sub28' 'Sub38'
 'Sub40']
--------------------------------
FOLD 1  Test Subjects: ['Sub01' 'Sub09' 'Sub10' 'Sub14' 'Sub18' 'Sub25' 'Sub26' 'Sub32' 'Sub34'
 'Sub35']
--------------------------------
FOLD 2  Test Subjects: ['Sub02' 'Sub03' 'Sub04' 'Sub06' 'Sub12' 'Sub22' 'Sub30' 'Sub31' 'Sub36'
 'Sub37']
--------------------------------
FOLD 3  Test Subjects: ['Sub08' 'Sub11' 'Sub15' 'Sub19' 'Sub21' 'Sub23' 'Sub24' 'Sub29' 'Sub33'
 'Sub39']


[I 2024-04-21 11:59:16,032] Trial 19 finished with value: 0.05336158365632097 and parameters: {'learning_rate': 0.0004905234731795299, 'weight_decay': 0.00014652213089607399, 'hidden_dim': 70, 'layer_dim': 4}. Best is trial 14 with value: 0.04780314459155004.


,number,value,datetime_start,datetime_complete,duration,params_hidden_dim,params_layer_dim,params_learning_rate,params_weight_decay,state
14,14,0.047803,2024-04-21 11:52:18.053594,2024-04-21 11:53:28.140486,0 days 00:01:10.086892,72,4,0.000748,0.000056,COMPLETE
1,1,0.048608,2024-04-21 11:38:35.407922,2024-04-21 11:39:48.109931,0 days 00:01:12.702009,25,4,0.001208,0.000033,COMPLETE
15,15,0.048699,2024-04-21 11:53:28.142188,2024-04-21 11:54:32.007659,0 days 00:01:03.865471,11,3,0.000896,0.000117,COMPLETE
10,10,0.049172,2024-04-21 11:47:40.649799,2024-04-21 11:48:51.297945,0 days 00:01:10.648146,63,4,0.000083,0.000013,COMPLETE
12,12,0.049788,2024-04-21 11:50:01.796340,2024-04-21 11:51:14.283338,0 days 00:01:12.486998,76,4,0.000129,0.000071,COMPLETE
11,11,0.050258,2024-04-21 11:48:51.299749,2024-04-21 11:50:01.794477,0 days 00:01:10.494728,64,4,0.000084,0.000011,COMPLETE
2,2,0.050638,2024-04-21 11:39:48.111659,2024-04-21 11:41:00.999952,0 days 00:01:12.888293,88,4,0.002310,0.000042,COMPLETE
17,17,0.052755,2024-04-21 11:55:45.956623,2024-04-21 11:56:58.880016,0 days 00:01:12.923393,78,4,0.009231,0.000034,COMPLETE
19,19,0.053362,2024-04-21 11:58:03.831605,2024-04-21 11:59:16.032485,0 days 00:01:12.200880,70,4,0.000491,0.000147,COMPLETE
16,16,0.054032,2024-04-21 11:54:32.009583,2024-04-21 11:55:45.949646,0 days 00:01:13.940063,96,4,0.000927,0.000219,COMPLETE


In [ ]:
best_trial_both = study_both.best_trial
best_trial_emg = study_emg.best_trial
best_trial_angle = study_kinematics.best_trial

best_trial_params = {}
best_trial_params['both'] = study_both.best_trial.params
best_trial_params['emg'] = study_emg.best_trial.params
best_trial_params['angle'] = study_kinematics.best_trial.params

best_trial_params['both']['num_epoch'] = 200
best_trial_params['emg']['num_epoch'] = 200
best_trial_params['angle']['num_epoch'] = 200


best_trial_dict = {}
best_trial_dict['best_trial_both'] = best_trial_both
best_trial_dict['best_trial_emg'] = best_trial_emg
best_trial_dict['best_trial_angle'] = best_trial_angle

best_trial_dict['study_both'] = study_both
best_trial_dict['study_emg'] = study_emg
best_trial_dict['study_kinematics'] = study_kinematics

Run all three experiment with their own best hyperparameter values

In [ ]:
# Set fixed random number seed
torch.manual_seed(42)

# Configuration options
k_folds = 4
# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

df_results_overall = {'use_both':{}, 'use_angle':{}, 'use_emg':{}, 'test_subjs_in_fold':{}}

subjects = data_selected.SubjID.unique()

# K-fold Cross Validation model evaluation
for fold, (train_subjs, test_subjs) in enumerate(kfold.split(subjects)):
    # Print
    print('--------------------------------')
    print(f'FOLD {fold}  Test Subjects: {subjects[test_subjs]}')

    train_idxs = data_selected[data_selected.SubjID.isin(subjects[train_subjs])].index
    test_idxs = data_selected[data_selected.SubjID.isin(subjects[test_subjs])].index

    train_df = data_selected.loc[train_idxs]
    test_df = data_selected.loc[test_idxs]
    df_results_overall['test_subjs_in_fold']['fold_'+str(fold)] = subjects[test_subjs]

    if 'use_both' in df_results_overall.keys():
        in_cols = emg_cols_preprocessed + kinematic_cols_preprocessed
        out_cols = kinetic_cols_preprocessed

        df_results = train_test(best_trial_params['both'], in_cols, out_cols, train_df, test_df)
        df_results = get_error_metrics(df_results, data_selected)
        df_results_overall['use_both']['fold_'+str(fold)] = df_results
        print('use_both end')

    if 'use_angle' in df_results_overall.keys():
        in_cols = kinematic_cols_preprocessed
        out_cols = kinetic_cols_preprocessed

        df_results = train_test(best_trial_params['angle'], in_cols, out_cols, train_df, test_df)
        df_results = get_error_metrics(df_results, data_selected)
        df_results_overall['use_angle']['fold_'+str(fold)] = df_results
        print('use use_angle end')

    if 'use_emg' in df_results_overall.keys():
        in_cols = emg_cols_preprocessed
        out_cols = kinetic_cols_preprocessed

        df_results = train_test(best_trial_params['emg'], in_cols, out_cols, train_df, test_df)
        df_results = get_error_metrics(df_results, data_selected)
        df_results_overall['use_emg']['fold_'+str(fold)] = df_results
        print('use_emg end')

    # Process is complete.
    print('Training process has finished for fold: ', fold)


In [ ]:
import pickle
with open('/content/drive/MyDrive/siat_dataset/outputs/experiment_vars_all_kfold_stance_0421_best.obj', 'wb') as fp:
    pickle.dump([data_selected, df_results_overall, best_trial_params, best_trial_dict], fp)